In [2]:
import glob, os
import xarray, wrf
from scipy import fftpack
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
fs=14
plt.rc("font",size=fs)

In [3]:
import numpy.matlib

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
outpath = "/glade/scratch/doubrawa/post_processing_new/"
inpath  = "/glade/scratch/doubrawa/final_data/"

In [6]:
kmax = 41
filter_meters = 333.*5
kappa_cut = 1/float(filter_meters)

In [7]:
ref_wrfnc   = Dataset("/glade/scratch/doubrawa/final_data/les/032815/032815_14UTC/wrfout_d04_2015-03-28_14:00:00_0000")

ref_patched = xarray.open_dataset("/glade/scratch/doubrawa/post_processing/WRF_LES_25m_SFC_2015-03-30_23:40.nc")

xlat_les    = ref_patched["lat"]
xlon_les    = ref_patched["lon"]

x_les, y_les = wrf.ll_to_xy(ref_wrfnc, xlat_les, xlon_les)                                

x_les   = np.reshape(x_les.data,xlat_les.shape)
y_les   = np.reshape(y_les.data,xlat_les.shape)

xmin, xmax = np.min(x_les), np.max(x_les)
ymin, ymax = np.min(y_les), np.max(y_les)

### the hamming filter only needs to be created once -- create it now

In [8]:
def return_hamming(Nx,Ny):

    # combo of uniform window in the middle and hamming window on edges:
    npts_buffer = 16

    # create a 1-d hamming window : hamm_1d.shape = (100,)
    hamm_1d     = np.hamming(npts_buffer*2)
    # repeat it : B.shape = (100, 100)
    B           = np.matlib.repmat(hamm_1d,npts_buffer*2,1)
    # transpose it : C.shape = (100,100)
    C           = np.transpose(B)
    # now get the two-dimensional hamming window : hamm_2d_s.shape = (100,100)
    hamm_2d_s   = B*C

    # allocate space for the final 2d-filter
    hamm_2d = np.zeros([Ny,Nx])

    # fill it with ones (no filter) anywhere that the window won't be applied (inside the domain, anywhere inside the buffer)        
    for ii in range(0+npts_buffer,Nx-npts_buffer):
        for jj in range(0+npts_buffer,Ny-npts_buffer):
            hamm_2d[jj,ii] = 1.0

    # now put the filter values in there

    # south west corner
    hamm_2d[0:npts_buffer,0:npts_buffer] = hamm_2d_s[0:npts_buffer,0:npts_buffer]

    # south east corner
    hamm_2d[0:npts_buffer,Nx-npts_buffer:Nx] = hamm_2d_s[0:npts_buffer,2*npts_buffer-npts_buffer:2*npts_buffer]

    # north west corner
    hamm_2d[Ny-npts_buffer:Ny,0:npts_buffer] = hamm_2d_s[2*npts_buffer-npts_buffer:2*npts_buffer,0:npts_buffer]

    # north east corner
    hamm_2d[Ny-npts_buffer:Ny,Nx-npts_buffer:Nx] = hamm_2d_s[2*npts_buffer-npts_buffer:2*npts_buffer,2*npts_buffer-npts_buffer:2*npts_buffer]

    # south boundary
    hann_tmp = hamm_1d[0:npts_buffer]
    len_tmp  = Nx-2*npts_buffer
    hann_tmp = np.matlib.repmat(hann_tmp,len_tmp,1)
    hann_tmp = np.transpose(hann_tmp)
    hamm_2d[0:npts_buffer,npts_buffer:Nx-npts_buffer] = hann_tmp

    # north boundary
    hann_tmp = hamm_1d[npts_buffer:2*npts_buffer]
    len_tmp  = Nx-2*npts_buffer
    hann_tmp = np.matlib.repmat(hann_tmp,len_tmp,1)
    hann_tmp = np.transpose(hann_tmp)
    hamm_2d[Ny-npts_buffer:Ny,npts_buffer:Nx-npts_buffer] = hann_tmp

    # west boundary
    hann_tmp = hamm_1d[0:npts_buffer]
    len_tmp  = Ny-2*npts_buffer
    hann_tmp = np.matlib.repmat(hann_tmp,len_tmp,1)
    hamm_2d[npts_buffer:Ny-npts_buffer,0:npts_buffer] = hann_tmp

    # east boundary
    hann_tmp = hamm_1d[npts_buffer:2*npts_buffer]
    len_tmp  = Ny-2*npts_buffer
    hann_tmp = np.matlib.repmat(hann_tmp,len_tmp,1)
    hamm_2d[npts_buffer:Ny-npts_buffer,Nx-npts_buffer:Nx] = hann_tmp
    
    return hamm_2d

In [ ]:
npts_befor_filt = 399
npts_after_filt = 90

In [ ]:
hamm_2d = return_hamming(Nx=npts_befor_filt,Ny=npts_befor_filt)

### prepare a butterworth filter

In [ ]:
wrf_dx = 333.

n_ord = 12 # order of the Butterworth filter
x_len = npts_befor_filt
y_len = npts_befor_filt
kappas_1dx = fftpack.fftfreq(x_len,d=wrf_dx)
kappas_1dy = fftpack.fftfreq(y_len,d=wrf_dx)
[kx_2d,ky_2d] = np.meshgrid(kappas_1dx,kappas_1dy)
k_xy_2d = np.sqrt(np.power(kx_2d,2.0)+np.power(ky_2d,2.0))
BWnth_2d_f = 1.0/(np.sqrt(1.0+(np.power(k_xy_2d/kappa_cut,2*n_ord))))

In [ ]:
sims  = ['YSU']#,'VLES','MYNN','YSU','SH']

count = 0

for sim in sims:
    
    print(sim)

    sim = sim.lower()

    daydirs = sorted(glob.glob(os.path.join(inpath,sim.lower(),'*')))
    
    for daydir in daydirs:
        
        wrfout_paths = sorted(glob.glob(os.path.join(daydir,'*d03*')))
                
        for wrfout_path in wrfout_paths:
            
            print(os.path.split(wrfout_path)[-1])
        
            data     = xarray.open_dataset(wrfout_path)
            wrfnc    = Dataset(wrfout_path,'r')                         
            
            if count==0:
                xlat  = wrf.getvar(wrfnc, "lat")
                xlon  = wrf.getvar(wrfnc, "lon")
                
                x, y  = wrf.ll_to_xy(ref_wrfnc, xlat, xlon)                
                
                x   = np.reshape(x.data,xlat.shape)
                y   = np.reshape(y.data,xlat.shape)

                condition_x = ( (x >= xmin) & (x<=xmax) )
                condition_y = ( (y >= ymin) & (y<=ymax) )
                condition   = condition_x & condition_y

                idx_sn, idx_we = np.where(condition)
                idx_sn = np.unique(idx_sn)
                idx_we = np.unique(idx_we)
    
                xlat  = xlat[idx_sn,idx_we]
                xlon  = xlon[idx_sn,idx_we]   
            
                x_clipped     = (x[idx_sn,idx_we]).copy()
                y_clipped     = (y[idx_sn,idx_we]).copy() 
                
                xlat.attrs = {} 
                xlon.attrs = {} 
                
                terrain  = wrf.getvar(wrfnc, "ter").isel(west_east=idx_we,south_north=idx_sn)                                
                terrain.attrs = {}
         
            #.isel(west_east=idx_we,south_north=idx_sn)
            z   = wrf.getvar(wrfnc, "z").squeeze().isel(bottom_top=range(kmax)).isel(west_east=idx_we,south_north=idx_sn)
            z   = (z-terrain).copy()
            
            u   = wrf.getvar(wrfnc, "ua").squeeze().isel(bottom_top=range(kmax))
            v   = wrf.getvar(wrfnc, "va").squeeze().isel(bottom_top=range(kmax))
            w   = wrf.getvar(wrfnc, "wa").squeeze().isel(bottom_top=range(kmax))
            T   = wrf.getvar(wrfnc, "theta").squeeze().isel(bottom_top=range(kmax))
                
            u.attrs = {}                
            v.attrs = {}                            
            w.attrs = {}                
            z.attrs = {}                
            T.attrs = {}                
                
            nk, nj, ni = u.data.shape
            u_f = np.zeros((nk,nj,ni))
            v_f = np.zeros((nk,nj,ni))
            w_f = np.zeros((nk,nj,ni))
            T_f = np.zeros((nk,nj,ni))

            ik = 0
            for k in range(nk):
                
                # 1. get 2-d arrays
                u_tmp = u.isel(bottom_top=k).data
                v_tmp = v.isel(bottom_top=k).data
                w_tmp = w.isel(bottom_top=k).data
                T_tmp = T.isel(bottom_top=k).data

                # 2. remove spatial mean
                u_tmp_mean = np.mean(u_tmp)
                v_tmp_mean = np.mean(v_tmp)
                w_tmp_mean = np.mean(w_tmp)
                T_tmp_mean = np.mean(T_tmp)                
                
                u_tmp = u_tmp - u_tmp_mean
                v_tmp = v_tmp - v_tmp_mean
                w_tmp = w_tmp - w_tmp_mean
                T_tmp = T_tmp - T_tmp_mean                

                # 3. apply window (to minimize noise since the arrays are not periodic in space)
                u_tmp = u_tmp * hamm_2d
                v_tmp = v_tmp * hamm_2d
                w_tmp = w_tmp * hamm_2d
                T_tmp = T_tmp * hamm_2d                

                # 4. apply fft
                u_psd_2d          = fftpack.fft2(u_tmp)
                v_psd_2d          = fftpack.fft2(v_tmp)
                w_psd_2d          = fftpack.fft2(w_tmp)
                T_psd_2d          = fftpack.fft2(T_tmp) 

                # 5. apply butterworth filter (instead of sharp filter, to minimize ringing effect near cut-off wavenumber)
                u_psd_2d = np.squeeze(u_psd_2d)*BWnth_2d_f
                v_psd_2d = np.squeeze(v_psd_2d)*BWnth_2d_f
                w_psd_2d = np.squeeze(w_psd_2d)*BWnth_2d_f
                T_psd_2d = np.squeeze(T_psd_2d)*BWnth_2d_f                

                # 6. invert to get the array back and put it in the kth index of the previously allocated 3-d array
                u_f[k,...]        = fftpack.ifft2(u_psd_2d).real + u_tmp_mean         
                v_f[k,...]        = fftpack.ifft2(v_psd_2d).real + v_tmp_mean         
                w_f[k,...]        = fftpack.ifft2(w_psd_2d).real + w_tmp_mean         
                T_f[k,...]        = fftpack.ifft2(T_psd_2d).real + T_tmp_mean         
                
            u_filt = xarray.DataArray(u_f, coords=u.coords,dims=u.dims)
            v_filt = xarray.DataArray(v_f, coords=v.coords,dims=v.dims)
            w_filt = xarray.DataArray(w_f, coords=w.coords,dims=w.dims)
            T_filt = xarray.DataArray(T_f, coords=T.coords,dims=T.dims)

            # clip
            u_filt = u_filt.isel(west_east=idx_we,south_north=idx_sn)
            v_filt = v_filt.isel(west_east=idx_we,south_north=idx_sn)
            w_filt = w_filt.isel(west_east=idx_we,south_north=idx_sn)
            T_filt = T_filt.isel(west_east=idx_we,south_north=idx_sn)  
            
            u = u.isel(west_east=idx_we,south_north=idx_sn)
            v = v.isel(west_east=idx_we,south_north=idx_sn)
            w = w.isel(west_east=idx_we,south_north=idx_sn)
            T = T.isel(west_east=idx_we,south_north=idx_sn)
            
            datetime    = pd.Timestamp(data.XTIME.data[0])
            attrs_dict  = {"valid":"{0:%Y-%m-%d_%H:%M}".format(datetime)}              
            xarray_dict = {"terrain":terrain,"xlat":xlat, "xlong":xlon, "z":z, "u":u, "v":v, "w":w, "theta":T,"u_filt":u_filt, "v_filt":v_filt, "w_filt":w_filt, "theta_filt":T_filt}
            dataset = xarray.Dataset(data_vars=xarray_dict,attrs=attrs_dict)            
            
            dataset = dataset.drop(["XTIME","Time"])
            dataset = dataset.assign_coords(south_north=dataset.south_north,west_east=dataset.west_east, bottom_top=dataset.bottom_top)
            dataset = dataset.rename({"XLONG":"lon","XLAT":"lat"}) 
            
            fout = "WRF_{0}_3D_{1:%Y-%m-%d_%H:%M}_plus_filtered.nc".format(sim,datetime)
            fout = os.path.join(outpath, fout)
            print("Saving : {0}".format(fout))
            dataset.to_netcdf(fout)

YSU
wrfout_d03_2015-03-20_12:10:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_12:10_plus_filtered.nc
wrfout_d03_2015-03-20_12:20:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_12:20_plus_filtered.nc
wrfout_d03_2015-03-20_12:30:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_12:30_plus_filtered.nc
wrfout_d03_2015-03-20_12:40:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_12:40_plus_filtered.nc
wrfout_d03_2015-03-20_12:50:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_12:50_plus_filtered.nc
wrfout_d03_2015-03-20_13:00:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_13:00_plus_filtered.nc
wrfout_d03_2015-03-20_13:10:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3D_2015-03-20_13:10_plus_filtered.nc
wrfout_d03_2015-03-20_13:20:00
Saving : /glade/scratch/doubrawa/post_processing_new/WRF_ysu_3